#Prérequis



*   Avoir un compte ngrok sur https://dashboard.ngrok.com
*   récupérer le numéro de token



#Installation de streamlit et ngrok

In [1]:
!pip install streamlit

     |████████████████████████████████| 9.7 MB 8.1 MB/s 
     |████████████████████████████████| 164 kB 48.9 MB/s 
     |████████████████████████████████| 4.3 MB 40.9 MB/s 
     |████████████████████████████████| 76 kB 5.2 MB/s 
     |████████████████████████████████| 111 kB 58.8 MB/s 
     |████████████████████████████████| 181 kB 51.4 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 128 kB 58.2 MB/s 
     |████████████████████████████████| 792 kB 53.7 MB/s 
     |████████████████████████████████| 380 kB 62.5 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=318d650adb9ea99af56ec0284f61f60d11a9fb780187c4d577a9f62cb79f792b
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      S

     |████████████████████████████████| 745 kB 8.6 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=a1a222a9dd83a8c7726cc5d5448330fe5d98b2b1fe9203607f4295e3bf8b2908
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!pip install pyngrok

# Développement de st_voice-sepration.py

On pourrait également faire !wget https://raw.github....st_voice-separation.py au mieu de l'écrire ici

In [12]:
%%writefile st_voice-separation.py
import pandas as pd
import streamlit as st

@st.cache
def get_dataviz_csv():
    url = "https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/data/liste_musdb18.csv"
    return pd.read_csv(url)

def get_all_scores_csv():
    url = "https://raw.githubusercontent.com/MickaRiv/ProjetDatascientest-VoiceSeparator/main/data/all_scores.csv"
    return pd.read_csv(url)

st.title("st.title")
st.header("st.header")
st.subheader("st.subheader")
st.markdown("st.markdown **gras**")
text_input = st.text_input('Ceci est un text_input :', 'Exemple text_input')
st.write('Ceci est un simple st.write')
st.caption("Ceci est du st.caption")
st.text("Ceci est du st.text")
st.code("Ceci est du st.code")
st.latex("\int a x^2\,df")

st.markdown("**Visu du dataframe liste_musdb18 du git :**")
df = get_dataviz_csv()
st.write(df)
st.dataframe(df)

st.write("**Visu du dataframe all_scores du git :**")
df2 = get_all_scores_csv()
st.write(df2)

#from PIL import Image
#image = Image.open('G:\Mon Drive\image.jpg')

#st.image(image, caption='Ceci est une image')


Overwriting st_voice-separation.py


#Lancement du streamlit

Start ngrock

In [56]:
!ls
!ngrok authtoken 25QRE4uYk7AG52BigElvRGk5TZc_4EfgeeiWXnGXYY2Qf5axB
!ngrok

sample_data  st_voice-separation.py
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com 

In [57]:
!streamlit run --server.port 80 st_voice-separation.py &>/dev/null& # en background

In [58]:
from pyngrok import ngrok
# Setup u tunnel to the streamlit port 8501
public_url = ngrok.connect(port='80')
public_url

<NgrokTunnel: "http://ce67-35-227-160-22.ngrok.io" -> "http://localhost:80">

# Arrêt de streamlit

Récupération de l'ID du process

In [53]:
!ps -eaf | grep streamlit

root         983       1  2 11:17 ?        00:00:02 /usr/bin/python3 /usr/local/bin/streamlit run --server.port 80 st_voice-separation.py
root        1019      71  0 11:19 ?        00:00:00 /bin/bash -c ps -eaf | grep streamlit
root        1021    1019  0 11:19 ?        00:00:00 grep streamlit


Kill du process

In [54]:
!kill 983

Arrêt de ngrok

In [55]:
ngrok.kill()